In [1]:
from cryptography.fernet import Fernet
import os

In [2]:
os.urandom(32)

b'\xc3F\xcf\x93\xbd\r\xd2L??\xc2K\xafE\xbf\xb9@\xea\xd3\xeb\xf4\x9b\xb5t4\x88\x9f\x84\xfdU$('

In [38]:
rand_1MB = os.urandom(1_048_576) # Wygeneruj losowy 1MB

with open("randoms.bin", "wb") as file: # Zapisz do pliku
    file.write(rand_1MB)

rand_1MB[:100]

In [2]:
def write_key():
    """
    Tworzy nowy klucz i zapisuje go do pliku.
    Zwraca stworzony klucz.
    """
    key = Fernet.generate_key()
    with open("key_file.key", "wb") as key_file:
        key_file.write(key)
    return key

In [2]:
def read_key():
    """
    Zwraca odczytany klucz z pliku.
    """
    try:
        file = open("key_file.key","rb")
        key = file.read()
        file.close()
        return key
    except Exception:
        print("Couldn't open key file!")
        return None

In [ ]:
key = read_key() # Użyj wartości b'Ww4Lpyluw0_XwOIFyIDA85mjCqjm0D2oQrDzftD43oc=' dla uploads.
key

In [4]:
def encrypt_data(data:bytearray, key:bytearray):
    """
    Zaszyfrowuje dane przekazane przez `data` z użyciem klucza przekazanego jako `key`.
    """
    f = Fernet(key)
    return f.encrypt(data)

In [5]:
def decrypt_data(data:bytearray, key:bytearray):
    """
    Odszyfrowuje dane przekazane przez `data` z użyciem klucza przekazanego jako `key`.
    """
    f = Fernet(key)
    return f.decrypt(data)

In [6]:
def encrypt_file(filename:str, key:bytearray):
    """
    Zaszyfrowuje plik o nazwie `filename` z użyciem klucza `key`, a zaszyfrowane dane zapisuje w pliku `filename + '.enc'`
    """
    try:
        infile = open(filename, "rb")
        enc_file = open(filename + ".enc", "wb")
        enc_file.write(encrypt_data(infile.read(),key))
        infile.close()
        enc_file.close()
        return True
    except Exception as e:
        print(e)
        return False

In [32]:
def decrypt_file(filename:str, key:bytearray):
    """
    Odszyfrowuje dane z pliku `filename` z użyciem klucza `key`, a odszyfrowane dane zapisuje do pliku `filename - '.enc'` dla plików których nazwa kończy się na `.enc`.
    W przeciwnym wypadku zapisuje dane do pliku o nazwie `filename + '.dec'`
    """
    try:
        enc_file = open(filename, "rb")
        if filename.lower().endswith(".enc"):
            filename = filename[:-4]
        else: filename += ".dec"
        dec_file = open(filename, "wb")
        dec_file.write(decrypt_data(enc_file.read(),key))
        dec_file.close()
        enc_file.close()
        return True
    except Exception as e:
        print(e)
        return False

In [33]:
encrypt_file("2022-10-25 19-54-59.mkv", key) # ~476MB -> 10.1s i5-4460

True

In [21]:
decrypt_file("2022-10-25 19-54-59.mkv.enc", key)# ~634MB (zaszyfrowane 476MB) -> 8.6s i5-4460

True

In [34]:
encrypt_file("2022-11-18 15-23-16.mkv", key) # ~1.09GB -> 27.7s i5-4460 (8GB RAM peak)

True

In [35]:
decrypt_file("2022-11-18 15-23-16.mkv.enc", key) # ~1.46GB -> 22.1s i5-4460 (4GB RAM peak)

True

In [7]:
encrypt_file("ca.crt",key)
encrypt_file("client.PJ.crt",key)
encrypt_file("client.PJ.key",key)

True

In [1]:
import sqlite3

In [3]:
db = sqlite3.connect("users.db")
cursor = db.cursor()
result = cursor.execute("SELECT * FROM files")

In [4]:
for res in result:
    print(res)

(1, 'e51102d100e99295d87bb245ed277581f37b34791cacd1fec1e6a34acdbeb574', 'jakub', 'Instrumenty_Patnicze_w_prawie_polskim.pdf', '10696297bcff003c666234efcc802b6c5118fe2e52e033e117424ad57ff61558')


In [5]:
cursor.close()
db.close()